In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
# Path to the zip file
zip_path = '/content/drive/MyDrive/Audio.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

In [ ]:
import tensorflow as tf
import numpy as np
import librosa
import os

In [ ]:
# Set constants
AUDIO_DIR = '/content/dataset/Audio/'
SAMPLE_RATE = 22050
MFCC_DIM = 13
N_CLUSTERS = 4


In [ ]:
# Load audio files
audio_files = [os.path.join(AUDIO_DIR, f) for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')]

In [ ]:
# Extract MFCC features from audio files
mfccs = []
max_len = 0
for file in audio_files:
    audio, sr = librosa.load(file, sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=MFCC_DIM)
    max_len = max(max_len, mfcc.shape[1])
    mfccs.append(mfcc)

In [ ]:
mfccs = []
max_len = 0
for file in audio_files:
    audio, sr = librosa.load(file, sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=MFCC_DIM)
    max_len = max(max_len, mfcc.shape[1])
    mfccs.append(mfcc)

In [ ]:

# Pad the MFCC arrays to a fixed length
mfccs_padded = []
for mfcc in mfccs:
    pad_width = ((0, 0), (0, max_len - mfcc.shape[1]))
    mfcc_padded = np.pad(mfcc, pad_width, mode='constant')
    mfccs_padded.append(mfcc_padded)

In [ ]:
# Convert the list of padded MFCC arrays to a 2D NumPy array
mfccs_padded_array = np.array(mfccs_padded)

In [ ]:
n_samples, n_mfcc, n_frames = mfccs_padded_array.shape
mfccs_padded_array = mfccs_padded_array.reshape(n_samples, n_mfcc * n_frames)


In [ ]:
scaler = StandardScaler()
mfccs_scaled = scaler.fit_transform(mfccs_padded_array)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=N_CLUSTERS)
labels = kmeans.fit_predict(mfccs_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Map cluster labels to traffic conditions
traffic_conditions = {
    0: "Free Flow",
    1: "Light Congestion",
    2: "Moderate Congestion",
    3: "Heavy Congestion"
}

In [ ]:
# Print the traffic conditions and corresponding audio files
for i, label in enumerate(labels):
    print(f"Traffic Condition: {traffic_conditions[label]} - Audio File: {audio_files[i]}")

Traffic Condition: Light Congestion - Audio File: /content/dataset/Audio/00005_1.wav
Traffic Condition: Light Congestion - Audio File: /content/dataset/Audio/00054_1.wav
Traffic Condition: Free Flow - Audio File: /content/dataset/Audio/00050_1.wav
Traffic Condition: Moderate Congestion - Audio File: /content/dataset/Audio/00022_1.wav
Traffic Condition: Free Flow - Audio File: /content/dataset/Audio/00053_1.wav
Traffic Condition: Moderate Congestion - Audio File: /content/dataset/Audio/00033_1.wav
Traffic Condition: Moderate Congestion - Audio File: /content/dataset/Audio/00028_1.wav
Traffic Condition: Free Flow - Audio File: /content/dataset/Audio/00044_1.wav
Traffic Condition: Light Congestion - Audio File: /content/dataset/Audio/00055_1.wav
Traffic Condition: Free Flow - Audio File: /content/dataset/Audio/00021_1.wav
Traffic Condition: Moderate Congestion - Audio File: /content/dataset/Audio/00016_1.wav
Traffic Condition: Moderate Congestion - Audio File: /content/dataset/Audio/00024